# VoxCity Demo — Solar Irradiance (Advanced Parameters)

This notebook demonstrates computing instantaneous and cumulative global solar irradiance using `voxcity.simulator.solar.get_global_solar_irradiance_using_epw`, with advanced parameters for control and performance.


In [ ]:
# Installation (uncomment if running in a fresh environment)
# %pip install voxcity
# %pip install contextily osmnx geopandas matplotlib timezonefinder


In [ ]:
import ee
from voxcity.geoprocessor.draw import draw_rectangle_map_cityname, center_location_map_cityname
from voxcity.generator import get_voxcity
from voxcity.simulator.solar import get_global_solar_irradiance_using_epw

# Authenticate & initialize Earth Engine (only needed if you plan to download data via GEE in your pipeline)
# ee.Authenticate()
# ee.Initialize(project='your-project-id')

cityname = "Tokyo, Japan"
meshsize = 5

# Option A: Draw rectangle interactively around a city
# m, rectangle_vertices = draw_rectangle_map_cityname(cityname, zoom=15)
# m  # display the map, draw a rectangle, then capture rectangle_vertices

# Option B: Click to set center with fixed width/height (meters)
# m, rectangle_vertices = center_location_map_cityname(cityname, east_west_length=500, north_south_length=500, zoom=15)
# m

# Option C: Provide rectangle vertices directly (lon, lat) if known
rectangle_vertices = [
    (139.760, 35.680),  # SW
    (139.760, 35.690),  # NW
    (139.770, 35.690),  # NE
    (139.770, 35.680)   # SE
]


In [ ]:
building_source = 'OpenStreetMap'
land_cover_source = 'OpenStreetMap'
canopy_height_source = 'High Resolution 1m Global Canopy Height Maps'
dem_source = 'DeltaDTM'

kwargs = {
    "output_dir": "output/solar_demo",
    # Optional DEM smoothing when reading GeoTIFF sources internally
    "dem_interpolation": True,
}

voxcity_grid, building_height_grid, building_min_height_grid, \
    building_id_grid, canopy_height_grid, canopy_bottom_height_grid, land_cover_grid, dem_grid, \
    building_gdf = get_voxcity(
        rectangle_vertices,
        building_source,
        land_cover_source,
        canopy_height_source,
        dem_source,
        meshsize,
        **kwargs
)

print(voxcity_grid.shape, dem_grid.shape)


## Instantaneous irradiance — core usage
Compute global solar irradiance for a specific time using EPW data. You can auto-download the nearest EPW based on the rectangle center.


In [ ]:
solar_kwargs = {
    "download_nearest_epw": True,
    "rectangle_vertices": rectangle_vertices,
    # Or set a local EPW directly:
    # "epw_file_path": "./output/your_city.epw",
    "calc_time": "01-01 12:00:00",
    "view_point_height": 1.5,
    "tree_k": 0.6,
    "tree_lad": 1.0,
    "dem_grid": dem_grid,
    "colormap": "magma",
    "obj_export": True,
    "output_directory": "output/solar_demo",
    "output_file_name": "instantaneous_solar_irradiance",
    "alpha": 1.0,
    "vmin": 0,
    # "vmax": 900,
}

solar_grid = get_global_solar_irradiance_using_epw(
    voxcity_grid, meshsize, calc_type='instantaneous',
    direct_normal_irradiance_scaling=1.0, diffuse_irradiance_scaling=1.0,
    **solar_kwargs
)

solar_grid.shape


## Cumulative irradiance — advanced windowing and performance
You can restrict hours of the day, set start/end time, and control threading to trade accuracy for speed on large time ranges.


In [ ]:
cum_kwargs = solar_kwargs.copy()
# Define the time window for accumulation
cum_kwargs["start_time"] = "01-01 05:00:00"
cum_kwargs["end_time"] = "01-31 20:00:00"

# Optionally restrict hours each day (e.g., 8AM–4PM)
cum_kwargs["start_hour"] = 8
cum_kwargs["end_hour"] = 16

# Performance controls (internal numba threads)
cum_kwargs["numba_num_threads"] = 4
cum_kwargs["progress_report"] = True

cum_kwargs["output_file_name"] = "cumulative_solar_irradiance"

cum_solar_grid = get_global_solar_irradiance_using_epw(
    voxcity_grid, meshsize, calc_type='cumulative',
    direct_normal_irradiance_scaling=1.0, diffuse_irradiance_scaling=1.0,
    **cum_kwargs
)

cum_solar_grid.shape


## Tips
- If EPW download fails due to SSL, consider `allow_insecure_ssl=True` or `allow_http_fallback=True` in kwargs.
- To avoid repeated downloads, set `download_nearest_epw=False` and provide `epw_file_path`.
- Adjust `tree_k` and `tree_lad` to tune vegetation transmittance.
